In [2]:
import torch
import numpy as np

## Elastic Constitutive Relation

#### Elastic Constitutive Relation Equation
Used to calculate the stress vectors.

\begin{equation} \tag{2}
\sigma = \left\{ \begin{array}{c} 
    \sigma_{xx} \\
    \sigma_{yy} \\
    \tau_{xy}
\end{array} \right\}
= \frac{E}{1 - \nu^2} \left[ \begin{array}{ccc}
    1 & \nu & 0 \\
    \nu & 1 & 0 \\
    0 & 0 & \frac{1 - v}{2}
    \end{array}    
\right] \left\{ \begin{array}{c} 
    \epsilon_{xx} \\
    \epsilon_{yy} \\
    \gamma_{xy}
\end{array} \right\}
\end{equation}

In the case of the inverse problem. Both $E$ and $\nu$ are predicted values. As a result, they themselves are tensors making the calculations more difficult.

In the example code `elastnet.py`, $\nu$ was assumed to be a constant 0.5, which made the matrix multiplication simpler. First is a method to quickly set test values.
Then is the provided code for a constant $\nu$ (`strain_c`). Lastly is a code for
calculating strain given a predicted $\nu$ (`strain_v`). 

The code for `strain_v` is likely not the optimal solution. There could be a
function that doesn't rely on list comprehension, or a method using built in
functions of torch. This needs to be looked in to. The difficulty is in creating
a matrix for each discrete point given the $\nu$ at that point.

In [33]:
# Test values
e_xx = torch.tensor(np.array([1., 1.7, 1.2,]), dtype=torch.float32)
e_yy = torch.tensor(np.array([2., 2.3, 2.1]), dtype=torch.float32)
r_xy = torch.tensor(np.array([3., 3.13, 3.2]), dtype=torch.float32)
strain= torch.stack([e_xx, e_yy, r_xy], dim = 1)

v_dat = torch.tensor(np.array([0.5, 0.5, 0.5]), dtype=torch.float32)
v_stack = torch.stack([v_dat, v_dat, v_dat], dim=1)

pred_E = torch.tensor([0.5, 1.0, 1.5], dtype=torch.float32)
modulus = torch.stack([pred_E, pred_E, pred_E], dim=1)

#### Code provided ($\nu$ = 0.5)

In [63]:
c_matrix = torch.tensor(
    (1/(1-(1/2.0)**2))*np.array([[1, 1/2.0, 0], [1/2.0, 1, 0], [0, 0, (1-1/2.0)/2.0]])
, dtype=torch.float32)

stress_c = torch.multiply(torch.matmul(strain, c_matrix), modulus)
stress_c

tensor([[1.3333, 1.6667, 0.5000],
        [3.8000, 4.2000, 1.0433],
        [4.5000, 5.4000, 1.6000]])

#### Calculating each value individually testing

In [35]:
def c_matrix(v:float):
    return torch.tensor(
        np.array(
            [ [1, v, 0], 
              [v, 1, 0], 
              [0, 0, (1-v)/2.0] ]
        ), dtype=torch.float32)

mat_res = torch.stack([ 
    torch.matmul(strain[i], c_matrix(v_stack[i, 0])) 
        for i in range(strain.shape[0]) 
])

# print(strain[0])
# print(c_matrix(v_stack[0,0]))
print(mat_res)

v2 = torch.square(v_stack)
fraction = torch.divide(modulus, 1 - v2)
stress_v = torch.multiply(mat_res, fraction)
stress_v

tensor([[2.0000, 2.5000, 0.7500],
        [2.8500, 3.1500, 0.7825],
        [2.2500, 2.7000, 0.8000]])


tensor([[1.3333, 1.6667, 0.5000],
        [3.8000, 4.2000, 1.0433],
        [4.5000, 5.4000, 1.6000]])

#### Stacking in a direction

In [62]:
c_stack = torch.stack([
    torch.ones(v_dat.shape, dtype=torch.float32),
    v_dat,
    torch.zeros(v_dat.shape, dtype=torch.float32), ##
    v_dat,
    torch.ones(v_dat.shape, dtype=torch.float32),
    torch.zeros(v_dat.shape, dtype=torch.float32), ##
    torch.zeros(v_dat.shape, dtype=torch.float32),
    torch.zeros(v_dat.shape, dtype=torch.float32),
    torch.divide(
        (torch.ones(v_dat.shape) - v_dat),
        torch.full(v_dat.shape, 2.0, dtype=torch.float32)
    ) ##
], dim=1).reshape([-1, 3,3])
# print(c_stack)

mat_res = torch.stack([ 
    torch.matmul(strain[i], c_stack[i]) 
        for i in range(strain.shape[0]) 
])

strain2 = strain.reshape(-1,1,3)
mat_res2 = torch.bmm(strain2, c_stack).squeeze()
print(mat_res2)
"""
GOAL:
tensor([[2.0000, 2.5000, 0.7500],
        [2.8500, 3.1500, 0.7825],
        [2.2500, 2.7000, 0.8000]])
"""

v2 = torch.square(v_stack)
fraction = torch.divide(1, 1 - v2) #modulus
stress_v = torch.multiply(mat_res2, fraction)
"""
GOAL:
tensor([[1.3333, 1.6667, 0.5000],
        [3.8000, 4.2000, 1.0433],
        [4.5000, 5.4000, 1.6000]])
"""
stress_v

tensor([[2.0000, 2.5000, 0.7500],
        [2.8500, 3.1500, 0.7825],
        [2.2500, 2.7000, 0.8000]])


tensor([[2.6667, 3.3333, 1.0000],
        [3.8000, 4.2000, 1.0433],
        [3.0000, 3.6000, 1.0667]])

In [53]:
# Displays the difference in the values
# If there is any, it was of magnitude e-16 for float around 1.0
# Display Just the differences
stress_v[(stress_v != stress_c)] - stress_c[(stress_v != stress_c)]
# Display all the differences (including 0)
torch.abs(stress_c - stress_v) # > 1e-15 # To filter out low values

IndexError: too many indices for tensor of dimension 2

## Equilibrium Condition

#### Sum of the sub stresses
Using a sliding 3x3 kernel

In [32]:
def conv2d(x, W):
    return torch.conv2d(x, W, strides = [1, 1, 1, 1], padding = 'VALID')

sum_kernel = np.array(
    [[[[1.0]], [[1.0]], [[1.0]]], 
     [[[1.0]], [[1.0]], [[1.0]]],
     [[[1.0]], [[1.0]], [[1.0]]], ]   
)

# Reshaped into 256 by 256 matrix
# Then shaped into 4d (-1, 256, 256, -1) for conv2d
# pred_E

conv2d(pred_E_4d, sum_kernel)

In [33]:
# Transform Stress to 4d (-1, 256, 256, -1)
stress_xx = stress[:, 0]
stress_yy = stress[:, 1]
stress_xy = stress[:, 2]

In [ ]:
# Convolutions from paper - calculate derivatives of strain
wx_conv_xx = np.array(
    [[[[-1.0]], [[-1.0]], [[-1.0]]], 
     [[[0.0]], [[0.0]], [[0.0]]],
    [[[1.0]], [[1.0]], [[1.0]]], ])
wx_conv_xy = np.array(
    [[[[1.0]], [[0.0]], [[-1.0]]], 
     [[[1.0]], [[0.0]], [[-1.0]]],
    [[[1.0]], [[0.0]], [[-1.0]]], ])
wy_conv_yy = np.array(
    [[[[1.0]], [[0.0]], [[-1.0]]], 
     [[[1.0]], [[0.0]], [[-1.0]]],
    [[[1.0]], [[0.0]], [[-1.0]]], ])
wy_conv_xy = np.array(
    [[[[-1.0]], [[-1.0]], [[-1.0]]], 
     [[[0.0]], [[0.0]], [[0.0]]],
    [[[1.0]], [[1.0]], [[1.0]]], ])

# Make tensors
wx_conv_xx = torch.tensor(wx_conv_xx, dtype = torch.float32)
wx_conv_xy = torch.tensor(wx_conv_xy, dtype = torch.float32)
wy_conv_yy = torch.tensor(wy_conv_yy, dtype = torch.float32)
wy_conv_xy = torch.tensor(wy_conv_xy, dtype = torch.float32)

In [ ]:

# From equilibrium condition
fx_conv_xx = conv2d(stress_xx_matrix_4d, wx_conv_xx)
fx_conv_xy = conv2d(stress_xy_matrix_4d, wx_conv_xy)
fx_conv_sum = fx_conv_xx + fx_conv_xy # Result that should be 0

fy_conv_yy = conv2d(stress_yy_matrix_4d, wy_conv_yy)
fy_conv_xy = conv2d(stress_xy_matrix_4d, wy_conv_xy)
fy_conv_sum = fy_conv_yy + fy_conv_xy # Result that should be 0

# Normalization, maybe h or t in e(i,j) equation
fx_conv_sum_norm = torch.divide(fx_conv_sum, pred_E_conv)
fy_conv_sum_norm = torch.divide(fy_conv_sum, pred_E_m_conv)

## Actually Calculating Loss

In [ ]:
# The potentially arbitrary constant
E_data = 1 # From data file
mean_modulus = torch.mean(E_data)

In [ ]:
# Equilibrium loss. PDE loss.
loss_x = torch.mean(torch.abs(fx_conv_sum_norm))
loss_y = torch.mean(torch.abs(fy_conv_sum_norm))

In [ ]:
# loss_m (modulus)
# L_e (13)
# "In practice mean value could be arbitrary > 0"
# To avoid minimizing E to 0, therefore use E_bar
# king of cheating in a way, could try and find a way to fix this
# Could apply boundary condition / include boundary condition
loss_m = torch.abs(torch.mean(pred_E) - mean_modulus)

In [ ]:
# DISPLACEMENT NOT POISSON, not used in inverse problem
# L_d (15)
#loss_v = torch.abs(torch.mean(y_pred_v))

In [ ]:
# Loss
# (12), loss_x + loss_y is loss_r
loss = loss_x + loss_y + loss_m/100

## Error (for reporting only)

In [ ]:
err_E = torch.sum(torch.abs(E_data - pred_E))
err_v = torch.sum(torch.abs(v_data - pred_v))

## Training
Uses an AdamOptimizer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=learn_rate)